In [ ]:
import numpy as np
import random
from scipy import stats
import matplotlib
import matplotlib.pyplot as plt
import deepRD
from deepRD.diffusionIntegrators import langevinNoiseSampler, langevin
from deepRD.potentials import harmonic
from deepRD.noiseSampler import noiseSampler
from deepRD.noiseSampler import binnedData
import deepRD.tools.trajectoryTools as trajectoryTools
matplotlib.rcParams.update({'font.size': 15})

In [ ]:
localDataDirectory = './data/'
numSimulations = 100 #100

In [ ]:
# Load binning sampling models
binnedDataFilename = localDataDirectory + 'qiBinnedData.pickle'
#binnedDataFilename = localDataDirectory + 'riBinnedData.pickle'
dataOnBins = pickle.load(open(binnedDataFilename, "rb" ))
parameters = dataOnBins.parameterDictionary
print(parameters)

In [ ]:
# Define particle list
D = parameters['D']
#position = [0,0,0]
#velocity = [0,0,0]
mass =  parameters['mass']
#particle1 = deepRD.particle(position, D, velocity, mass)
#particleList = deepRD.particleList([particle1])
#position = position = np.array([boxsize*random.random()-0.5*boxsize,
#                                    boxsize*random.random()-0.5*boxsize,
#                                    boxsize*random.random()-0.5*boxsize])

In [ ]:
# Define noise sampler
#nSampler = noiseSampler()
nSampler = noiseSampler(dataOnBins)

In [ ]:
# Define external potential
kconstant = 0.3
harmonicPotential = harmonic(kconstant)

In [ ]:
# Define integrator
#dt = binnedData.lagTimesteps * parameters['dt'] * parameters['stride'] #wrong
dt = parameters['dt']
integratorStride = 50 #5
tfinal = 1000 #10000 #0.5 #1 #1000 #100 #10000 #100000
KbT = parameters['KbT']
boxsize = parameters['boxsize']
#integType = "ABOBA"
equilibrationSteps = 10000# 50000 #10000 #2000
diffIntegrator = langevinNoiseSampler(dt, integratorStride, tfinal, nSampler, KbT, 
                                      boxsize, equilibrationSteps = equilibrationSteps)
#diffIntegrator = langevin(dt, integratorStride, tfinal, KbT, boxsize, equilibrationSteps = equilibrationSteps)
diffIntegrator.setExternalPotential(harmonicPotential)

In [ ]:
particleIndex = 0
xpos = np.empty(0)
ypos = np.empty(0)
zpos = np.empty(0)
xvel = np.empty(0)
yvel = np.empty(0)
zvel = np.empty(0)
trajs = []
for i in range(numSimulations):
    print("Simulation ", i+1)
    position = [0,0,0] #np.array([boxsize*random.random()-0.5*boxsize,boxsize*random.random()-0.5*boxsize,boxsize*random.random()-0.5*boxsize])
    velocity = [0,0,0]
    particle1 = deepRD.particle(position, D, velocity, mass)
    particleList = deepRD.particleList([particle1])
    t, X, V = diffIntegrator.propagate(particleList)
    traj = trajectoryTools.convert2trajectory(t,[X,V])
    trajs.append(traj)
    xpos = np.concatenate((xpos, X[:,particleIndex,0]))
    ypos = np.concatenate((ypos, X[:,particleIndex,1]))
    zpos = np.concatenate((zpos, X[:,particleIndex,2]))
    xvel = np.concatenate((xvel, V[:,particleIndex,0]))
    yvel = np.concatenate((yvel, V[:,particleIndex,1]))
    zvel = np.concatenate((zvel, V[:,particleIndex,2]))
    print("")

In [ ]:
numbins=50
#averagepos = (np.array(xpos)+np.array(ypos)+np.array(zpos))/3.0
#plt.hist(averagepos, bins = numbins, density= True, alpha =0.3)
plt.hist(xpos, bins = numbins, density= True, alpha=0.5, label='x')
plt.hist(ypos, bins = numbins, density= True, alpha=0.5, label='y')
plt.hist(zpos, bins = numbins, density= True, alpha=0.5, label='z')
plt.legend()
#plt.xlim([-0.15,0.15])
plt.show()

In [ ]:
plt.plot(xpos)

In [ ]:
numbins=50
#averagepos = (np.array(xpos)+np.array(ypos)+np.array(zpos))/3.0
#plt.hist(averagepos, bins = numbins, density= True, #alpha =0.3)
plt.hist(xvel, bins = numbins, density= True, alpha=0.5, label='x')
plt.hist(yvel, bins = numbins, density= True, alpha=0.5, label='y')
plt.hist(zvel, bins = numbins, density= True, alpha=0.5, label='z')
plt.legend()
#plt.xlim([-0.15,0.15])
plt.show()

## Load one benchmark trajectory

In [ ]:
import deepRD.tools.trajectoryTools as trajectoryTools
import os

In [ ]:
parentDirectory = os.environ.get('MSMRD') + '/data/MoriZwanzig/benchmark_onetrajectory/'
fnamebase = parentDirectory + 'simMoriZwanzig_'

In [ ]:
# Load trajectory data from h5 files (only of distinguished particle)
trajs_ref = []
print("Loading data ...")
for i in range(numSimulations):
    traj = trajectoryTools.loadTrajectory(fnamebase, i)
    trajs_ref.append(traj)    
    print("File ", i+1, " of ", numSimulations, " done.", end="\r")

In [ ]:
# Plot distributions
xpos_ref = []
ypos_ref = []
zpos_ref = []
xvel_ref = []
yvel_ref = []
zvel_ref = []
numbins = 50
for traj in trajs_ref:
    for i in range(len(traj)):
        xpos_ref.append(traj[i][1])
        ypos_ref.append(traj[i][2])
        zpos_ref.append(traj[i][3])
        xvel_ref.append(traj[i][4])
        yvel_ref.append(traj[i][5])
        zvel_ref.append(traj[i][6])
#averagepos = (np.array(xpos_ref)+np.array(ypos_ref)+np.array(zpos_ref))/3.0
#plt.hist(averagepos, bins = numbins, density= True, alpha =0.3)
plt.hist(np.array(xpos_ref), bins = numbins, density= True, alpha=0.5, label='x')
plt.hist(np.array(ypos_ref), bins = numbins, density= True, alpha=0.5, label='y')
plt.hist(np.array(zpos_ref), bins = numbins, density= True, alpha=0.5, label='z')
plt.legend()
plt.show()

In [ ]:
plt.hist(np.array(xvel_ref), bins = numbins, density= True, alpha=0.5, label='x')
plt.hist(np.array(yvel_ref), bins = numbins, density= True, alpha=0.5, label='y')
plt.hist(np.array(zvel_ref), bins = numbins, density= True, alpha=0.5, label='z')
plt.legend()
plt.show()

## Comparison

In [ ]:
numbins=50
plt.hist(np.array(yvel), bins = numbins, density= True, alpha=0.5, label='reduced')
plt.hist(np.array(yvel_ref), bins = numbins, density= True, alpha=0.5, label='benchmark')
density = stats.kde.gaussian_kde(xvel)
#vv = np.arange(-1., 1, .02)
#plt.plot(vv, density(vv))
plt.legend()
plt.xlabel("velocity")
plt.ylabel("velocity distribution")
#plt.show()
plt.savefig("velocity_test.pdf", bbox_inches='tight')

In [ ]:
numbins=50
#plt.hist(np.array(zpos), bins = numbins, density= True, alpha=0.5, label='reduced')
plt.hist(np.array(zpos_ref), bins = numbins, density= True, alpha=0.5, label='benchmark')
plt.legend()
plt.xlabel("position")
plt.ylabel("position distribution")
plt.xlim([-5,5])
plt.show()
#plt.savefig("position_test.pdf", bbox_inches='tight')

## Calculate autocorrelations

In [ ]:
# Functions to calculate moments
def calculateMean(trajs, var = 'position'):
    if var == 'position':
        index = 1
    elif var == 'velocity':
        index = 4
    meanPosition = np.zeros(3)
    totalSamples = 0
    for traj in trajs:
        for i in range(len(traj)):
            meanPosition += traj[i][index:index+3]
        totalSamples += len(traj)
    meanPosition = meanPosition/totalSamples
    return meanPosition

def calculateVariance(trajs, var = 'position', mean = None):
    if var == 'position':
        index = 1
    elif var == 'velocity':
        index = 4
    if mean.any() == None:
        mean = calculateMean(trajs)
    variance = np.zeros(3)
    totalSamples = 0
    for traj in trajs:
        for i in range(len(traj)):
            devFromMean = traj[i][index:index+3] - mean
            variance += devFromMean*devFromMean
        totalSamples += len(traj)
    variance = variance/totalSamples
    return variance

def calculateStdDev(trajs, var = 'position', mean = None):
    variance = calculateVariance(trajs, var, mean)
    stddev = np.sqrt(variance)
    return stddev

def calculateAutoCorrelation(trajs, lagtimesteps, stride = 1, var = 'position', mean = None, variance = None):
    if var == 'position':
        index = 1
    elif var == 'velocity':
        index = 4
    if mean.any() == None:
        mean = calculateMean(trajs)
    if variance.any() == None:
        variance = calculateVariance(trajs, mean)
    totalSamples = 0
    AC = 0.0
    for traj in trajs:
        for i in range(len(traj)-lagtimesteps*stride):
            devFromMean = traj[i][index:index+3] - mean
            devFromMean2 = traj[i + lagtimesteps*stride][index:index+3] - mean
            AC += np.dot(devFromMean, devFromMean2)
        totalSamples += len(traj)
    AC = AC/totalSamples
    AC = AC/variance
    return AC

def calculateAutoCorrelationFunction(trajs, lagtimesteps, stride = 1, var = 'position'):
    ACF = []
    mean = calculateMean(trajs, var)
    # Calculate one dimensional variance
    if var == 'position':
        index = 1
    elif var == 'velocity':
        index = 4
    variance = 0
    totalSamples = 0
    for traj in trajs:
        for i in range(len(traj)):
            devFromMean = traj[i][index:index+3] - mean
            variance += np.dot(devFromMean,devFromMean)
        totalSamples += len(traj)
    variance = variance/totalSamples
    for lagtime in range(lagtimesteps):
        ACF.append(calculateAutoCorrelation(trajs, lagtime, stride, var, mean, variance))
        print('Computing ACF:', 100*(lagtime+1)/lagtimesteps , '% complete   ', end="\r")
    ACF = np.array(ACF)
    return ACF

In [ ]:
variables = ['position', 'velocity']
lagtimesteps = 40
strides = [50,1]
ACF = [None]*2
ACF_ref = [None]*2
for i, var in enumerate(variables):
    mean = calculateMean(trajs, var)
    mean_ref = calculateMean(trajs_ref, var)
    variance = calculateVariance(trajs, var, mean)
    variance_ref = calculateVariance(trajs_ref, var, mean_ref)
    ACF[i] = calculateAutoCorrelationFunction(trajs, lagtimesteps, strides[i], var)
    ACF_ref[i] = calculateAutoCorrelationFunction(trajs_ref, lagtimesteps, strides[i], var)

In [ ]:
index = 1
time = dt*integratorStride*strides[index]*np.linspace(1,lagtimesteps,lagtimesteps)
plt.plot(time, ACF[index], 'xk', label = 'reduced model')
plt.plot(time, ACF_ref[index], '-k', label = 'benchmark')
plt.xlabel('time(ns)')
plt.ylabel(variables[index] + ' autocorrelation')
plt.legend()
#plt.xlim([0,1500])
plt.savefig(variables[index]+ ' autocorrelation.pdf')